In [434]:
import pandas as pd
from pulp import *
import requests
import warnings
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.font_manager as font_manager
import matplotlib as mpl
import datetime
warnings.filterwarnings('ignore')

In [435]:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
#Request packagae to make GET request from the API endpoint
r = requests.get(url)

#Transform request variable to a JSON object
json = r.json()
json.keys()

#Build a Dataframe
elements_df = pd.DataFrame(json['elements'])

#Checking the columns
elements_df.columns

#Getting the required metrics for analysis
red_elements_df = elements_df[['team','id','first_name','second_name','web_name','element_type','cost_change_start','now_cost','selected_by_percent', 'transfers_in', 'transfers_out', 'total_points', 'bonus', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'status', 'form']]

#Map Element type to actual position of the players
red_elements_df['element_type']=red_elements_df['element_type'].map({4:'Forward', 3:'Midfielder', 2:'Defender', 1:'Goalkeeper'})

#Map team names
red_elements_df['team']=red_elements_df['team'].map({1:'Arsenal',2:'Aston Villa',3:'Bournemouth',4:'Brentford',5:'Brighton',6:'Chelsea',7:'Crystal Palace',8:'Everton',9:'Fulham',10:'Leicester City',11:'Leeds United',12:'Liverpool',13:'Manchester City',14:'Manchester Utd',15:'Newcastle Utd',16:'Nottingham Forest',17:'Southampton',18:'Tottenham',19:'West Ham',20:'Wolves'})

#Convert value to float
red_elements_df['selected_by_percent'] = red_elements_df.selected_by_percent.astype(float)
red_elements_df['cost_change_start'] = red_elements_df['cost_change_start']/10
red_elements_df['now_cost'] = red_elements_df['now_cost']/10

#rename columns
red_elements_df.columns = ['Team','Player ID','First Name','Second Name','Web Name','Position','Start Price','Current Price','Selected By','Transfers In', 'Transfers Out', 'Total Points', 'Bonus', 'Minutes', 'Goals Scored', 'Assists', 'Clean Sheets', 'Status', 'Form']


In [436]:
df = red_elements_df.copy()

#Net Transfers
df['Net Transfers'] = df['Transfers In'] - df['Transfers Out']

#Form to float
df['Form'] = df['Form'].astype(float)
df.head()

,Team,Player ID,First Name,Second Name,Web Name,Position,Start Price,Current Price,Selected By,Transfers In,Transfers Out,Total Points,Bonus,Minutes,Goals Scored,Assists,Clean Sheets,Status,Form,Net Transfers
0,Arsenal,1,Cédric,Alves Soares,Cédric,Defender,-0.2,4.3,0.2,2447,12836,0,0,0,0,0,0,a,0.0,-10389
1,Arsenal,3,Granit,Xhaka,Xhaka,Midfielder,0.0,5.0,2.5,172317,43794,20,2,267,1,2,2,a,6.7,128523
2,Arsenal,4,Mohamed,Elneny,Elneny,Midfielder,-0.1,4.4,1.4,71991,64856,0,0,0,0,0,0,a,0.0,7135
3,Arsenal,5,Rob,Holding,Holding,Defender,-0.2,4.3,0.1,1420,7117,0,0,0,0,0,0,a,0.0,-5697
4,Arsenal,6,Thomas,Partey,Partey,Midfielder,0.0,5.0,0.6,22698,26331,8,0,270,0,0,2,a,2.7,-3633


In [437]:
df = df[['Web Name', 'Team', 'Position', 'Total Points', 'Current Price', 'Minutes', 'Net Transfers','Status', 'Form']]
df = df.rename(columns={'Web Name': 'Name', 'Total Points': 'Points', 'Current Price': 'Price', 'Net Transfers': 'NetTransfers'})

In [438]:

POS = df.Position.unique()
CLUBS = df.Team.unique()
BUDGET = 100


pos_available = {
    'Defender': 5,
    'Forward': 3,
    'Midfielder': 5,
    'Goalkeeper': 2,
}

# Initialize Variables
names = [df.Name[i] for i in df.index]
teams =  [df.Team[i] for i in df.index]
positions =  [df.Position[i] for i in df.index]
prices =  [df.Price[i] for i in df.index]
points =  [df.Points[i] for i in df.index]
minutes = [df.Minutes[i] for i in df.index]
transfers = [df.NetTransfers[i] for i in df.index]
status = [df.Status[i] for i in df.index]
form = [df.Form[i] for i in df.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in df.index]

In [439]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

In [440]:
# Build the constraints
# Define the objective
prob += lpSum(players[i] * transfers[i] for i in range(len(df))) # Objective

prob += lpSum(players[i] * df.Price[df.index[i]] for i in range(len(df))) <= BUDGET # Budget Limit


for pos in POS:
  prob += lpSum(players[i] for i in range(len(df)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(df)) if teams[i] == club) <= 3 # Club Limit

In [441]:
# Solve the problem
prob.solve()

1

In [442]:
#print results as a dataframe
df_result = pd.DataFrame(columns=['Name', 'Team', 'Position', 'Points', 'Price', 'NetTransfers', 'Status', 'Form'])
for v in prob.variables():
    if v.varValue != 0:
        name = df.Name[int(v.name.split("_")[1])]
        club = df.Team[int(v.name.split("_")[1])]
        position = df.Position[int(v.name.split("_")[1])]
        point = df.Points[int(v.name.split("_")[1])]
        price = df.Price[int(v.name.split("_")[1])]
        minutes = df.Minutes[int(v.name.split("_")[1])]
        transfers = df.NetTransfers[int(v.name.split("_")[1])]
        status = df.Status[int(v.name.split("_")[1])]
        form = df.Form[int(v.name.split("_")[1])]
        df_result = df_result.append({'Name': name, 'Team': club, 'Position': position, 'Points': point, 'Price': price, 'Minutes': minutes, 'NetTransfers': transfers, 'Status': status, 'Form': form}, ignore_index=True)

In [443]:

#Points per minute
df_result['Points per million'] = df_result['Points'] / df_result['Price']
df_result['Points per million'] = df_result['Points per million'].astype(float).round(2)
df_result['Price'] = df_result['Price'].round(2)
df_result['Minutes'] = df_result['Minutes'].astype(int)
df_result.sort_values(by=['Form', 'NetTransfers'], ascending=False, inplace=True)
df_result.reset_index(drop=True, inplace=True)
df_result

#reverse the order of the dataframe
df_result = df_result.iloc[::-1]

In [444]:
font_dir = ['/Users/vivektiwari/Code/FontsPoppins']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
mpl.rcParams['font.family'] = 'Poppins'

In [445]:
import themepy

theme = themepy.Theme()
(theme
.set_theme('neon')
.set_font('Poppins')
.set_pips(False)# we could set the tick marks on the x and y axis on or off here
.set_spines("off", which=["top","right","bottom","left"]) # we could show or hide the borders of the plot
.set_ticklabel_size() # we could change the size of the x and y tick labels here
.set_grid("off") # we could set the grid on or off and set params like color and linewidth
)

fig, ax = plt.subplots(figsize=(12, 8))

rows = df_result.shape[0]
cols = df_result.shape[1]

ax.set_ylim(-1, rows + 1)
ax.set_xlim(0, cols + .5)

for row in range(rows):
    ax.plot([0, cols + 1], [row -.5, row - .5], ls=':', lw='.5', c='grey')

for row in range(rows):
    d = df_result.iloc[row]

    ax.text(x =0.5, y = row, s=d['Name'], va ='center', ha = 'left')
    ax.text(x =2, y = row, s=d['Team'], va ='center', ha = 'left')
    ax.text(x =4, y = row, s=d['Position'], va ='center', ha = 'center')
    ax.text(x =5, y = row, s=d['Points'], va ='center', ha = 'center')
    ax.text(x =6, y = row, s=d['Price'], va ='center', ha = 'center')
    ax.text(x =7, y = row, s=d['Minutes'], va ='center', ha = 'center')
    ax.text(x =8.5, y = row, s=d['NetTransfers'], va ='center', ha = 'center')
    ax.text(x =10, y = row, s=d['Form'], va ='center', ha = 'center')

    #add column headers
    ax.text(0.8, 15, 'Name', va='center', ha='center', fontsize=11)
    ax.text(2.5, 15, 'Team', va='center', ha='center', fontsize=11)
    ax.text(4, 15, 'Position', va='center', ha='center', fontsize=11)
    ax.text(5, 15, 'Points', va='center', ha='center', fontsize=11)
    ax.text(6, 15, 'Price', va='center', ha='center', fontsize=11)
    ax.text(7, 15, 'Minutes', va='center', ha='center', fontsize=11)
    ax.text(8.5, 15, 'Net Transfers', va='center', ha='center', fontsize=11)
    ax.text(10, 15, 'Form', va='center', ha='center', fontsize=11)

    if row == 14:
        ax.plot([0, cols + 1], [row + .6, row + .6], ls='-', lw='1', c='#06AD55')
    
    if row == 3:
        ax.plot([0, cols + 1], [row + .55, row + .55], ls='-', lw='1', c='#6ACD98')

    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

ax.set_title('Kneejerk United - (Season 2022-23)', fontsize=16, fontweight='bold', loc = 'left', pad = 10)
ax.text(1.9, 16, 'Net Transfers = Transfers In - Transfers Out | Sorted by Form', va='center', ha='center', fontsize=8)
#date
ax.text(0.75, -1, 'Team created on: ' + str(datetime.date.today()), va='center', ha='center', fontsize=8)
ax.text(9.5, -1, 'Created by: @Stateastic', va='center', ha='center', fontsize=8)
    #save the plot as a png wth date
plt.savefig('Kneejerk_United_' + str(datetime.date.today()) + '.png', bbox_inches='tight', dpi=300)

